In [1]:
import numpy as np
import pandas as pd
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, show

import aframe_plots.plot_utils as utils

output_notebook()

Loading BokehJS ...

In [4]:
combos = [
    (35, 35),
    (35, 20),
    (20, 20),
    (20, 10)
]
plots = []
for i, combo in enumerate(combos):
    kwargs = dict(
        title=r"$$\text{{Log Normal }}m_1={}, m_2={}$$".format(*combo),
        x_axis_type="log",
        tools="save"
    )

    kwargs["width"] = 350
    if not i % 2:
        # plots on the left need space for y-axis label
        kwargs["width"] += 30
        kwargs["y_axis_label"] = r"$$\text{Sensitive Distance [Mpc]}$$"

    kwargs["height"] = 220
    if i > 1:
        # lower plots need space for x-axis label
        kwargs["height"] += 30
        kwargs["x_axis_label"] = r"$$\text{False Alarm Rate [months}^{-1}\text{]}$$"

    p = utils.get_figure(**kwargs)
    p.outline_line_color = "#ffffff"
    if i < 2:
        utils.hide_axis(p, "x")

    for i in range(3):
        color = utils.palette[i]
        df = pd.read_csv(f"data/uncertainty-comparison/run{i + 1}_sd-vs-far.csv")
        x = df["far"].values
        y = df["ln_{}_{}".format(*combo)]
        err = df["ln_{}_{}_err".format(*combo)]
        p.line(
            x,
            y,
            line_width=1.5,
            line_color=color
        )
        utils.plot_err_bands(
            p,
            x,
            y,
            err,
            line_color=color,
            line_width=0.8,
            fill_color=color,
            fill_alpha=0.4
        )

    plots.append(p)

grid = gridplot(plots, toolbar_location="right", ncols=2)
show(grid)